## Import modules and network graph

In [1]:
#%% Import modules
import os
import shapely.geometry
import pandas as pd
import uuid

# our software
import opentnsim.core as TNcore
import openclsim.model as CLmodel
import openclsim.core as CLcore

import dtv_backend.dtv_backend as backend


## Initialize environment with network

In [2]:
#%% Initialize environment
# set up environment
env = backend.provide_environment()

# load the network
backend.load_DTV_network_to_env(env)

Defining simulation environment
Starting (down)loading the network
Network succesfully added to simulation


## Define some locations to use

In [3]:
# some interesting locations
locations = {
    'Transferium Maasvlakte': shapely.geometry.Point(4.087406, 51.936737),
    'Neusse': shapely.geometry.Point(6.708892, 51.215737),
    'Basel': shapely.geometry.Point(7.640572, 47.555449),
    'Nijmegen': shapely.geometry.Point(5.8161152, 51.8570535),
    'Waal at St. Andries': shapely.geometry.Point(5.350339, 51.803965)
}

## Define sites 

In [4]:
#%% define sites
from dtv_backend.core import sites as backendSites

# define origin
origin = backendSites.provideSite(env = env, 
                           point = locations['Transferium Maasvlakte'], 
                           name = 'Transferium Maasvlakte',
                           capacity = 10000,
                           level = 10000,
                           loading_rate = 1000,
                           unloading_rate = 1000)

# define destination
destination = backendSites.provideSite(env = env, 
                           point = locations['Basel'], 
                           name = 'Basel',
                           capacity = 10000,
                           level = 0,
                           loading_rate = 1000,
                           unloading_rate = 1000)


## Determine minimal waterlevel based on lobith discharge

In [6]:
import pathlib
import dtv_backend
from dtv_backend.network import network_utilities

depth_path = pathlib.Path(os.path.join(dtv_backend.__path__[0], '..', 'data', 'depth.csv'))
discharge_df = pd.read_csv(depth_path)

lobith_discharge = 10000
min_waterdepth_on_path = network_utilities.determine_min_waterdepth_on_path(env.FG, origin, destination, discharge_df, lobith_discharge)

underkeel_clearance = 0.30
max_draught = min_waterdepth_on_path - underkeel_clearance

NodeNotFound: Source (22161408.0, 0.0) not in G

In [ ]:
min_waterdepth_on_path

## Create a vessel

In [ ]:
#%% define a vessel
from dtv_backend.core import vessels as backendVessels

data_110_a = {
    "env": env,
    "name": "NPRC_110",
    "geometry": origin.geometry,
    "loading_rate": 1,
    "unloading_rate": 1,
    "capacity": 3000,
    "allowable_draught": max_draught,
    "route": None,
    'vessel_type': 'M8',
    'installed_power': 1000,
    'width': 10, 
    'length': 110, 
    'height_empty': 8, 
    'height_full': 4, 
    'draught_empty': 2, 
    'draught_full': 6
}

NPRC_110_a = backendVessels.provideVessel(**data_110_a)


data_110_b = {
    "env": env,
    "name": "NPRC_110",
    "geometry": origin.geometry,
    "loading_rate": 1,
    "unloading_rate": 1,
    "capacity": 3000,
    "allowable_draught": max_draught,
    "route": None,
    'vessel_type': 'M8',
    'installed_power': 1000,
    'width': 10, 
    'length': 110, 
    'height_empty': 8, 
    'height_full': 4, 
    'draught_empty': 2, 
    'draught_full': 6
}

NPRC_110_b = backendVessels.provideVessel(**data_110_b)


## Functionality to create a single run process but with the moving functionalities of openTNsim

In [ ]:
from dtv_backend.processes.single_run_process_fleet import single_run_process

single_run_a, activity_a, while_activity_a  = single_run_process(
    name="single_run_a",
    registry={},
    env=env,
    origin=origin,
    destination=destination,
    mover=NPRC_110_a,
    loader=origin,
    unloader=destination
)

single_run_b, activity_b, while_activity_b  = single_run_process(
    name="single_run_b",
    registry={},
    env=env,
    origin=origin,
    destination=destination,
    mover=NPRC_110_b,
    loader=origin,
    unloader=destination
)

env.run()

In [ ]:
pd.DataFrame(NPRC_110_a.log)

In [ ]:
pd.DataFrame(NPRC_110_b.log)[]

In [ ]:
origin.container.get_level()

In [ ]:
# Export results 
import networkx as nx
import json
import shapely.wkt

import geopandas as gpd

import dtv_backend.postprocessing

# Find route from A to B
path = nx.dijkstra_path(env.network, origin_node, destination_node, weight='Length')

result_objs = {
    'origin': origin, 
    'destination': destination, 
    'equipment': NPRC_110
}

logs = {}
for key, obj in result_objs.items():
    log_gdf = gpd.GeoDataFrame(pd.DataFrame(NPRC_110.log), geometry='Geometry')
    log_gdf['Timestamp'] = log_gdf['Timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')
    logs[key] = json.loads(log_gdf.to_json())

path_gdf = dtv_backend.postprocessing.path2gdf(path, env.network)

sites = []
for site in [origin, destination]:
    obj = {
        key: getattr(site, key) 
        for key in vars(site) 
        if key not in ["env", "resource", "container", "log", "wgs84"]
    }
    sites.append(obj)
sites_gdf = gpd.GeoDataFrame(sites, geometry='geometry')
    
result["sites"] = json.loads(sites_gdf.to_json())
    
result["path"] = json.loads(path_gdf.to_json())
result["logs"] = logs


with open('sample-result.json', 'w') as f:
    json.dump(result, f)



## old code

In [ ]:
# stop before execuring old code
raise

In [ ]:
# Find basic route
start_point = locations_nodes['Transferium Maasvlakte']
end_point = locations_nodes['Nijmegen']
path = nx.dijkstra_path(env.FG, start_point, end_point, weight='Length')

# blockin at...
blocking = locations_nodes['Waal at St. Andries']

# assert that the blocking is in the route
print(f'The blocking is in the path: {blocking in path}')
assert blocking in path, 'Blocking should be in the shortest path'

# quick check on the graph: do the edges have resources?
edge = list(env.FG.edges(blocking))[0]
print(f'Edges have resources? {"Resources" in env.FG.edges[edge].keys()}')

In [ ]:
# get all edges connecting to St. Andries
blocked_edges = []
for edge in env.FG.edges():
    if blocking in edge:
        blocked_edges.append(edge)

# set a trigger (node) for the edge to be blocked 2/4 of the way towards the blocked node
index = int(path.index(blocking)*(2/4))
trigger_node = path[index]

In [ ]:
# Make a transport resource class out of mix-ins
TransportResource = type('TransportResource', 
                         (opentnsim.core.Identifiable, opentnsim.core.ContainerDependentMovable, 
                          opentnsim.core.HasResource, opentnsim.core.Routeable), {})

# For testing purposes we only want v to be 1, whether empty or loaded
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

# Define vessel data, but do not yet link to env or route
data_vessel = {"env": None,
               "name": "Vessel number 1",
               "route": None,
               "geometry": G.nodes[start_point]['geometry'],  # lon, lat
               "capacity": 1_000,
               "compute_v": compute_v_provider(v_empty=1, v_full=1)}


In [ ]:
# create the transport processing resource
vessel1 = TransportResource(**data_vessel)

# Add the vessel to the environment, and define the start and end point
vessel1.env = env
vessel1.start_loc = start_point
vessel1.end_loc = end_point

# also define its current location as the start point for now
vessel1.current_loc = start_point

# add the route
vessel1.route = path

# check route
#vessel1.route

In [ ]:
# create the transport processing resource
vessel2 = TransportResource(**data_vessel)

# Add the vessel to the environment, and define the start and end point
vessel2.env = env
vessel2.start_loc = start_point
vessel2.end_loc = end_point

# also define its current location as the start point for now
vessel2.current_loc = start_point

#--> seems as though this path is sailed even though it is updated on the while loop of the start function

# add the route (is it needed to add it as an empty list?)
#vessel2.route = []

# check route
#vessel2.route

In [ ]:
# simply move a vessel accross a predefined route
def start(env, vessel):
    yield from vessel.move()
    

In [ ]:
# function to sail vessel with route decision
def start_dynamic(env, vessel, blocked_edges, trigger_node):
    """ Have a vessel sail from start_point to end_point, while after every
    egde passed checking if the current route is still available"""
    # while vessel's current location is not end location
    while vessel.current_loc != vessel.end_loc:
        # activate blockage if triggered
        if vessel.current_loc == trigger_node:
            for blocked_edge in blocked_edges:
                env.FG.edges[blocked_edge]['Length'] = 999999
        # determine the shortest path from the current to the end location given the current conditions
        shortest_path = nx.dijkstra_path(env.FG, vessel.current_loc, vessel.end_loc, weight='Length')
        # make sure the vessel passes only the first edge in the route
        vessel.route = [shortest_path[0], shortest_path[1]]
        # make sure the current location is now updated
        vessel.current_loc = shortest_path[1]
        yield from vessel.move()
    

In [ ]:
# add the processes and run the environments
env.process(start(env, vessel1))
env.process(start_dynamic(env, vessel2, blocked_edges, trigger_node))
env.run()

In [ ]:
# check the log of vessel 1
df1 = pd.DataFrame.from_dict(vessel1.log)
df1.head(4).append(df1.tail(4))

In [ ]:
# check the log of vessel 2
df2 = pd.DataFrame.from_dict(vessel2.log)
df2.head(4).append(df2.tail(4))

In [ ]:
# create figure
fix, ax = plt.subplots()

# get log and plot track of vessel 1
df1['x'] = df1.Geometry.apply(lambda geom: geom.x)
df1['y'] = df1.Geometry.apply(lambda geom: geom.y)
df1.plot('x', 'y', label='vessel1', ax=ax)

# get log and plot track of vessel 2
df2['x'] = df2.Geometry.apply(lambda geom: geom.x)
df2['y'] = df2.Geometry.apply(lambda geom: geom.y)
df2.plot('x', 'y', label='vessel2', ax=ax)

In [ ]:
gdf1 = gpd.GeoDataFrame(df1.rename(columns={'Geometry':'geometry'}), crs="EPSG:4326")
gdf2 = gpd.GeoDataFrame(df2.rename(columns={'Geometry':'geometry'}), crs="EPSG:4326")

In [ ]:
import folium
import shapely
m = folium.Map(location=[52, 5], zoom_start=9)
col1 = '#ff0000'
col2 = '#3785b8'

style1 = {'fillColor': col1, 'color': col1, 'line_opacity': 0.2}
style2 = {'fillColor': col2, 'color': col2, 'opacity': 0.8}

folium.GeoJson(shapely.geometry.LineString(gdf1.geometry), style_function=lambda x:style1).add_to(m)
folium.GeoJson(shapely.geometry.LineString(gdf2.geometry), style_function=lambda x:style2).add_to(m)

folium.Marker([G.nodes[blocking]['Y'],G.nodes[blocking]['X']],
          icon = folium.Icon(icon='times', color='red', prefix='fa'),
          ).add_to(m)

m